# Make the validation plots and save them

### Set the folder where the results of the run are saved. The validation results will be also saved there

In [5]:
path= "/eos/home-f/frehm/TF2/Skripts/F02-03/Runs/run13/run13_4/run13_4/"    #path to the run folder
path= "/eos/home-f/frehm/TF2/Skripts/F02-03/pure tf/PTF_runs/PTF_8/PTF_1/"    #path to the run folder
#path= "/eos/home-f/frehm/TF2/Skripts/F02-03/Runs/Improving_Keras_Model/Imp_1/Imp_1/"
#path = os.getcwd() + "/run13_4/    #or place this file into the run folder
#path= "/eos/home-f/frehm/TF2/Skripts/F02-03/Runs/Run-up-to-date/run-test/"

run_name = "PTF_6"

from Functions_v1 import *
create_folder(path+ "/validation")

Directory  /eos/home-f/frehm/TF2/Skripts/F02-03/pure tf/PTF_runs/PTF_8/PTF_1//validation  Created 


## Create a .gif of the output images of the generator of each epoch

In [6]:
#create Gif
from PIL import Image, ImageDraw
import os

path_img= path + "Save_Images"
save_name = '/gif_' + run_name + '.gif'
images = []
listing = os.listdir(path_img)
#listing=listing[1:]

for i in listing:
    img = Image.open(path_img+ "/" + i)
    images.append(img)

images[0].save(path + "/validation" + save_name, save_all=True, append_images=images[1:], optimize=False, duration=500, loop=0)
print("gif successfull created")

gif successfull created


## Make the loss plots of the training and save them all in one PDF document

In [7]:
#Analysis loss plots
import os
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
try:
    import cPickle as pickle
except ImportError:
    import pickle
from matplotlib.backends.backend_pdf import PdfPages  #saving all img to one pdf

lossfile = path + '/Pickle/3dgan-history_dict.pkl'
x = pickle.load( open( lossfile, "rb" ) )
weights = [6, 0.2, 0.1]
#defining limits for different plots. Varies with result
ymax = [10, 3, 2, 40, 0.75, 1.] #[weighted loss for all, BCE_test loss, BCE_train_loss, Aux unweighted, BCE-train for generator only]

outdir = path + 'validation/loss_plots/'
validation_dir = path + "validation/"
create_folder(outdir)
print('The plots are saved in {}'.format(outdir))

fig = 1
gen_test =   np.asarray(x['test']['generator'])
gen_train =  np.asarray(x['train']['generator'])
disc_test =  np.asarray(x['test']['discriminator'])
disc_train = np.asarray(x['train']['discriminator'])

pp = PdfPages(validation_dir + 'validation_loss.pdf')   #saving all img to one pdf
#Plots for Testing and Training Losses
plt.figure(fig)
plt.subplot(221)
plt.title('Disc. Weighted Train loss')
plt.plot(disc_train[:,0], label='tot')
plt.plot(weights[0] * disc_train[:,1], label='gen')
plt.plot(weights[1] * disc_train[:,2], label='aux')
plt.plot(weights[2] * disc_train[:,3], label='ecal')
plt.legend()                                                                   
plt.ylim(0, ymax[0])                                                                                                                           

plt.subplot(222)
plt.title('Gen. Weighted Train loss')
plt.plot(gen_train[:,0], label='tot')
plt.plot(weights[0] * gen_train[:,1], label='gen')
plt.plot(weights[1] * gen_train[:,2], label='aux')
plt.plot(weights[2] * gen_train[:,3], label='ecal')
plt.legend()                                                                   
plt.ylim(0, ymax[0])                                                                                              

plt.subplot(223)
plt.title('Testing loss for Discriminator')
plt.plot(disc_test[:,0], label='tot')
plt.plot(weights[0] * disc_test[:,1], label='gen')
plt.plot(weights[1] * disc_test[:,2], label='aux')
plt.plot(weights[2] * disc_test[:,3], label='ecal')
plt.legend()
plt.ylim(0, ymax[0])  

plt.subplot(224)
plt.title('\nTesting loss for Generator')
plt.plot(gen_test[:,0], label='tot')
plt.plot(weights[0] * gen_test[:,1], label='gen')
plt.plot(weights[1] * gen_test[:,2], label='aux')
plt.plot(weights[2] *disc_test[:,3], label='ecal')
plt.legend()
plt.ylim(0, ymax[0])  
plt.savefig(os.path.join(outdir,'losses.png')) 
plt.savefig(pp, format='pdf')   #saving all img to one pdf

#Training losses
fig = fig + 1
plt.figure(fig)
plt.title('Weighted Training losses for GAN: Loss weights = (%0.2f, %.2f, %.2f)'%(weights[0], weights[1], weights[2]))
plt.plot(disc_train[:,0], label='Disc tot', color='red')
plt.plot(weights[0] * disc_train[:,1], label='Disc gen (Binary Cross Entropy)', color='green')
plt.plot(weights[1] * disc_train[:,2], label='Disc aux (Mean Absolute Percentage Error)', color='blue')
plt.plot(weights[2] * disc_train[:,3], label='Disc ecal(Mean Absolute Percentage Error)', color='magenta')
plt.plot(gen_train[:,0], label='Gen tot', color='red', linestyle='--')
plt.plot(weights[0] * gen_train[:,1], label='Gen gen (Binary Cross Entropy)', color='green', linestyle='--')
plt.plot(weights[1] * gen_train[:,2], label='Gen aux (Mean Absolute Percentage Error)', color='blue', linestyle='--')
plt.plot(weights[2] * gen_train[:,3], label='Gen ecal(Mean Absolute Percentage Error)', color='magenta', linestyle='--')
plt.legend()
plt.xlabel('Epochs')  
plt.ylabel('Loss')                            
plt.ylim(0, ymax[0])  
plt.savefig(os.path.join(outdir, 'Combined_losses.png'))
plt.savefig(pp, format='pdf')

#training losses for Real/fake
fig = fig + 1
plt.figure(fig)
plt.title('Binary Training losses for GAN')
plt.plot(disc_train[:,1], label='Disc gen (Binary Cross Entropy)', color='green')
plt.plot(gen_train[:,1], label='Gen gen (Binary Cross Entropy)', color='blue')
plt.legend()
plt.xlabel('Epochs')  
plt.ylabel('Loss')                            
plt.ylim(0, ymax[1])  
plt.savefig(os.path.join(outdir, 'BCE_train_losses.png'))
plt.savefig(pp, format='pdf')

#training losses for Real/fake Magnified
fig = fig + 1
start_epoch = 10
plt.figure(fig)
plt.title('Binary Training losses for Generator')
#plt.plot(disc_train[:,1], label='Disc gen (Binary Cross Entropy)', color='green')
plt.plot(gen_train[start_epoch:,1], label='Gen gen (Binary Cross Entropy)', color='blue')
plt.legend()
plt.xlabel('Epochs starting from epoch' + str(start_epoch))  
plt.ylabel('Loss')                            
plt.ylim(ymax[4], ymax[5])  
plt.savefig(os.path.join(outdir, 'BCE_train_gen_losses.png'))
plt.savefig(pp, format='pdf')

#testing losses for Real/fake
fig = fig + 1
plt.figure(fig)
plt.title('Binary Testing losses for GAN')
plt.plot(disc_test[:,1], label='Disc gen (Binary Cross Entropy)', color='green')
plt.plot(gen_test[:,1], label='Gen gen (Binary Cross Entropy)', color='blue')
plt.legend()
plt.xlabel('Epochs')  
plt.ylabel('Loss')                            
plt.ylim(0, ymax[2])  
plt.savefig(os.path.join(outdir, 'BCE_test_losses.png'))
plt.savefig(pp, format='pdf')

#Training losses for auxlilliary losses
fig = fig + 1
plt.figure(fig)
plt.title('Training losses for Auxilliary outputs')
plt.plot(disc_train[:,2], label='Disc aux (Mean Absolute Percentage Error)', color='blue')
plt.plot(disc_train[:,3], label='Disc ecal(Mean Absolute Percentage Error)', color='magenta')
plt.plot(gen_train[:,2], label='Gen aux (Mean Absolute Percentage Error)', color='blue', linestyle='--')
plt.plot(gen_train[:,3], label='Gen ecal(Mean Absolute Percentage Error)', color='magenta', linestyle='--')
plt.legend()
plt.xlabel('Epochs')  
plt.ylabel('Loss')                            
plt.ylim(0, ymax[3])  
plt.savefig(os.path.join(outdir, 'aux_training_losses.png'))
plt.savefig(pp, format='pdf')

#Diff. for training losses Real/fake
fig = fig + 1
plt.figure(fig)
plt.title('Binary Training losses for GAN')
plt.plot(gen_train[:,1] - disc_train[:,1], label='Gen loss - Disc loss (Binary Cross Entropy)')
plt.legend()
plt.xlabel('Epochs')  
plt.ylabel('Loss')                            
plt.ylim(0, ymax[1])  
plt.savefig(os.path.join(outdir, 'BCE_train_losses_diff.png'))
plt.savefig(pp, format='pdf')

pp.close()    #saving all img to one pdf
print("\nSaving loss plots successful")

Directory  /eos/home-f/frehm/TF2/Skripts/F02-03/pure tf/PTF_runs/PTF_8/PTF_1/validation/loss_plots/  Created 
The plots are saved in /eos/home-f/frehm/TF2/Skripts/F02-03/pure tf/PTF_runs/PTF_8/PTF_1/validation/loss_plots/

Saving loss plots successful
